In [2]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from konlpy.tag import Okt
from torch.utils.data import DataLoader
torch.manual_seed(777)
from sklearn.preprocessing import normalize

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
import pickle

with open("./pickle_data/balanced_data.pickle", "rb") as f:
    balenced_data = pickle.load(f)
    
with open("./pickle_data/token_data.pickle", "rb") as f:
    token_data = pickle.load(f)

In [7]:
comment = balenced_data["comment"]
target = balenced_data["target"]
reviews_idx = token_data["comment_ix"]
word2idx = token_data["word2ix"]
idx2word = token_data["ix2word"]
max_seq_length = token_data["max_seq_length"]
idx2target={1:'highlight',0:'no'}

In [9]:
a=[]
for i,k in zip(comment,target):
    a.append([i,k])

In [10]:
len(comment)

20343

In [11]:
b=[]
for i in range(0,len(comment)):
    if a[i][1]==1:
        b.append(a[i])

In [12]:
b[0][0]

['킹/Noun', '킹민철/Noun']

In [15]:
len(b)

9932

In [16]:
target_non=[]
for k in range(0,10622):
    target_non.append(0)

In [17]:
len(target_non)

10622

In [18]:
X_train,comment_non,y_train,y_test=train_test_split(li,target_non,test_size=0.3)

ValueError: Found input variables with inconsistent numbers of samples: [9932, 10622]

In [10]:
idx2rate={}
for i,k in zip(rating,rating_index):
    if k not in idx2rate:
        idx2rate[k]=i
    else:
        pass

In [11]:
okt=Okt()

In [12]:
token_review=[]
token_review+=[okt.morphs(i) for i in reviews]

In [13]:
len_token=[]
len_token+=[len(i) for i in token_review]

In [14]:
n=-1
for i in token_review:
    k=len(i)
    n+=1
    while max(len_token)-k>0:
        token_review[n].append('<pad>')
        k+=1

In [15]:
a=[]
a+=[ k for i in token_review for k in i]

In [16]:
bag_of_word=list(set(a))
word_index=lb.fit_transform(bag_of_word)

In [17]:
word2idx={}
idx2word={}
for i,k in zip(bag_of_word,word_index):
    word2idx[i]=k
    idx2word[k]=i

In [18]:
bag=[]
for i in token_review:
    li=[]
    
    for k in i:
        li.append(word2idx[k])
    bag.append(li)

In [19]:
rating_index

array([1, 1, 1, ..., 1, 0, 1])

In [20]:
X_train,X_test,y_train,y_test=train_test_split(bag,rating_index,test_size=0.3)

In [21]:
from torch.utils.data import Dataset

class makeData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data 
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [22]:
train_data=makeData(X_data=np.array(X_train),y_data=y_train)
test_data=makeData(X_data=np.array(X_test),y_data=y_test)

In [23]:
len(X_train)

1278

In [34]:
torch.FloatTensor(y_train)

tensor([1., 1., 1.,  ..., 0., 0., 1.])

In [24]:
len(X_test)

549

In [25]:
train_loader=DataLoader(dataset=train_data,batch_size=142,shuffle=True)
test_loader=DataLoader(dataset=test_data,batch_size=1,shuffle=False)

In [19]:
class LSTM_Clf(nn.Module):
    def __init__(self, input_size, embedding_size,num_class,num_layers,hidden_size,bidirectional,num_directions):
        
        super(LSTM_Clf,self).__init__()
        
        # 안에 하이퍼 파라미터의 종류들 선언
        
        self.num_layers=num_layers
        self.hidden_size=hidden_size
        self.input_size=input_size
        self.embedding_size=embedding_size
        self.num_class=num_class
        self.num_directions=num_directions
        
        self.embedding=nn.Embedding(input_size,
                                    embedding_size)
        
        self.lstm=nn.LSTM(input_size=self.embedding_size,
                        hidden_size=self.hidden_size,
                        num_layers=self.num_layers,
                        batch_first=True,
                        bidirectional=bidirectional) # 해당 sequence의 앞 뒤를 모두 고려해서 반영
        
        self.linear=nn.Linear(in_features=self.hidden_size*self.num_directions,out_features=self.num_class)

    
    def forward(self,inputs):
        
        hidden,cell=self.init_hidden(batch_size)
        
        embed=self.embedding(inputs)
        
        out, _ =self.lstm(embed,(hidden,cell))
        
        return self.linear(out[:, -1:, :].squeeze(1))
    
    
    def predict(self,inputs):
        
        hidden,cell = self.init_hidden(1)
        embed = self.embedding(inputs)

        # Propagate embedding through RNN
        # Input: (batch, seq_len, embedding_size)
        # hidden: (num_layers * num_directions, batch, hidden_size)
        
        out, _ = self.lstm(embed, (hidden,cell))
        out=self.linear(out[:, -1:, :].squeeze(1))
        
        return torch.round(out)
    
    def init_hidden(self, batch_size):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        
        hidden=Variable(torch.zeros(self.num_layers*self.num_directions,batch_size, self.hidden_size)).to(device)
        cell=Variable(torch.zeros(self.num_layers*self.num_directions,batch_size, self.hidden_size)).to(device)
        
        return hidden,cell
    
    

In [38]:
model = LSTM_Clf(input_size=7974,num_directions=1, embedding_size = 200,hidden_size=max(len_token),
                num_layers=3,num_class=1,bidirectional=False).to(device)

loss_function = nn.BCELoss().to(device)
optimizer = optim.Adam(model.parameters(),lr=0.01,weight_decay=0.1)

In [39]:
%%time
EPOCH=15
batch_size=142
for epoch in range(EPOCH):
    
    test_pred=[]
    test_true=[]
    
    model.train()
    for k,(X_batch,y_batch) in enumerate(train_loader):
       
        model.zero_grad()
        inputs=Variable(torch.Tensor(X_batch.long())).to(device)
        targets=Variable(torch.Tensor(y_batch.float())).to(device)
        
        y_pred=model(inputs)
        
        loss=loss_function(y_pred,targets)
        
        
        loss.backward()
        optimizer.step()
        
        if (k+1) % 1 ==0:
            print('epoch: [{}/{}] Step [{}/{}], Loss: {:.4f}'
                .format((epoch+1),EPOCH,(k+1),len(train_loader),loss))
    print('\n')        
    print('=====update finished!=====')
          
    test_pred=[]
    test_true=[]
    model.eval()
    with torch.no_grad():
        for k,(X_batch,y_batch) in enumerate(test_loader):
            inputs=Variable(X_batch).to(device)
            targets=Variable(y_batch).to(device)

            y_pred=model.predict(inputs)
            y_pred=torch.max(y_pred,1)[1]
            test_pred.append(y_pred.item())
            test_true.append(targets.item())

        print('test_acc: %.3f'%(accuracy_score(y_true=test_true,y_pred=test_pred)))
        print('test_f1 %.3f' %(f1_score(y_true=test_true,y_pred=test_pred)))
        print('\n') 


    print('------------------------------')

TypeError: expected torch.FloatTensor (got torch.LongTensor)